In [ ]:
### Input Processing
import pandas as pd
import numpy as np


np.random.seed(42)
data = pd.read_csv('../input/Kannada-MNIST/train.csv')

data['row_number'] = range(0,data.shape[0])


print(data.shape)
data = data.sample(frac=1,random_state=42)
tmp = pd.DataFrame()
for label in range(10):
    if label==0:
        tmp = data[data['label']==label].head(600)
    else:
        temp = data[data['label']==label].head(600)
        tmp = pd.concat([tmp,temp])
data_train = tmp
row_numbers_in_train_set = tmp['row_number'].values
test_set = data.loc[~data['row_number'].isin(row_numbers_in_train_set)]
print(data_train[['label']].groupby('label').size().reset_index())

one_hot = pd.get_dummies(data_train['label'].unique())
one_hot['label'] = one_hot.index

data_train = pd.merge(data_train,one_hot)
#data = data.drop('label',axis=1)
data_test = test_set.sample(frac=1)

tmp = pd.DataFrame()
for label in range(10):
    if label==0:
        tmp = data_test[data_test['label']==label].head(120)
    else:
        temp = data_test[data_test['label']==label].head(120)
        tmp = pd.concat([tmp,temp])
data_test = tmp
#data_test = data_test.head(500)
data_test = pd.merge(data_test,one_hot)
data_train.drop('label',axis=1,inplace=True)

data_test.drop('label',axis=1,inplace=True)

## Create the train and test set
X_train = np.array(data_train.drop([0,1,2,3,4,5,6,7,8,9,'row_number'],axis=1).values)/255
#X_train = (X_train - np.mean(X_train)) / np.std(X_train)
y_train = np.array(data_train[[0,1,2,3,4,5,6,7,8,9]].values)
X_test = np.array(data_test.drop([0,1,2,3,4,5,6,7,8,9,'row_number'],axis=1).values)/255
#X_test = (X_test - np.mean(X_test)) / np.std(X_test)
y_test = np.array(data_test[[0,1,2,3,4,5,6,7,8,9]].values)


In [ ]:
def maxpool(input_matrix,stride=2,filter_h=2,filter_w = 2,padding=0):
    relu = np.ones((input_matrix.shape[1],filter_h,filter_w))
    result_h = int((input_matrix.shape[2]-filter_h+padding)/stride)+1
    result_w = int((input_matrix.shape[3]-filter_w+padding)/stride)+1
    relu_result = np.zeros((input_matrix.shape[0],input_matrix.shape[1],result_h,result_w))
    max_indices_for_result_relu = []
    for image in range(input_matrix.shape[0]):
        for filter_position in range(relu.shape[0]):
            image_selected = input_matrix[image,:,:,:]
            #result = []
            filter_selected = relu[filter_position,:,:]
            image_selected = image_selected[filter_position,:,:]
            row = -1
            column = -1
            for i in range(0,image_selected.shape[0],stride):
                image_rectangle = image_selected[i:i+filter_h,:]
                if image_rectangle.shape[0]<filter_h:
                    continue
                else:
                    row = row+1
                    if row>result_h-1:
                        row = 0
                    for j in range(0,image_rectangle.shape[1],stride):
                        image_portion = image_rectangle[:,j:j+filter_w]
                        if image_portion.shape[1]<filter_w:
                            continue
                        else:
                            column = column+1
                            if column>result_w-1:
                                column = 0
                            convolution_result = np.multiply(filter_selected,image_portion)
                            convolution_result[convolution_result<0] = 0
                            i_max,j_max = np.unravel_index(convolution_result.argmax(),convolution_result.shape)
                            i_max=i_max+i
                            j_max = j_max+j
                            
                            max_indices_for_result_relu.append((image,filter_position,i_max,j_max,image,filter_position,row,column))                                          
                            relu_result[image,filter_position,row,column] = np.max(convolution_result)
                                
                                
            #result = np.array(result)
            #result = np.ravel(result)
            
            #relu_result[image,filter_position,:,:] = result.reshape(result_h,result_w)
    return(relu_result,max_indices_for_result_relu)


        

def maxpool_reshape(shape_,input_matrix):
    maxpool_reshape = np.zeros(shape_)
    for row in range(input_matrix.shape[0]):
        temp = input_matrix[row,:,:,:]
        temp = np.ravel(temp)
        maxpool_reshape[:,row] = temp
    return(maxpool_reshape)
    
    
    ### Function to reshape maxpool output to dense layer
def delta_conv_filter(filter_,input_matrix,delta_matrix,stride=1):
    conv_delta = np.zeros(filter_.shape)
    filter_h = filter_.shape[2]
    filter_w = filter_.shape[3]
    delta_x_conv = delta_matrix
    X = input_matrix
    for image in range(delta_x_conv.shape[0]):
        delta_image = delta_x_conv[image,:,:,:]
        input_image = X[image,:,:,:]
        for channel in range(delta_image.shape[0]):
            delta_image_channel = delta_image[channel,:,:]
            delta_values = np.ravel(delta_image_channel)
            for values in delta_values:
                #print(values)
                #### Get chunks from input matrix with same shape as filter
                for row in range(0,input_image.shape[1],stride):
                    image_rectangle = input_image[:,row:row+filter_h,:]
                    if image_rectangle.shape[1]<filter_h:
                        continue
                    else:
                        for column in range(0,image_rectangle.shape[2],stride):
                            image_square = image_rectangle[:,:,column:column+filter_w]
                            #print(image_square.shape)
                            if image_square.shape[2]<filter_w:
                                continue
                            else:
                                conv_delta[channel,:,:,:] +=(values*image_square)   
    return(conv_delta)
                       
def im2col(X,conv1, stride, pad):
    """
        Transforms our input image into a matrix.

        Parameters:
        - X: input image.
        - HF: filter height.
        - WF: filter width.
        - stride: stride value.
        - pad: padding value.

        Returns:
        -cols: output matrix.
    """
    # Padding
    X_padded = np.pad(X, ((0,0), (0,0), (pad, pad), (pad, pad)), mode='constant')
    X = X_padded
    new_height = int((X.shape[2]+(2*pad)-(conv1.shape[2]))/stride)+1
    new_width =  int((X.shape[3]+(2*pad)-(conv1.shape[3]))/stride)+1
    im2col_vector = np.zeros((X.shape[1]*conv1.shape[2]*conv1.shape[3],new_width*new_height*X.shape[0]))
    c = 0
    for position in range(X.shape[0]):

        image_position = X[position,:,:,:]
        for height in range(0,image_position.shape[1],stride):
            image_rectangle = image_position[:,height:height+conv1.shape[2],:]
            if image_rectangle.shape[1]<conv1.shape[2]:
                continue
            else:
                for width in range(0,image_rectangle.shape[2],stride):
                    image_square = image_rectangle[:,:,width:width+conv1.shape[3]]
                    if image_square.shape[2]<conv1.shape[3]:
                        continue
                    else:
                        im2col_vector[:,c:c+1]=image_square.reshape(-1,1)
                        c = c+1         
            
    return(im2col_vector)




In [ ]:
X_train = X_train.T
y_train = y_train.T

X_test = X_test.T
y_test = y_test.T

def initializeFilter(size, scale = 1.0):
    stddev = scale/np.sqrt(np.prod(size))
    return np.random.normal(loc = 0, scale = stddev, size = size)

def sigmoid(x):
    return(1./(1+np.exp(-x)))

def softmax(x): 
    """Compute softmax values for each sets of scores in x.""" 

    e_x = np.exp(x - np.max(x)) 

    return (e_x / e_x.sum(axis=0)) 
def ReLU(x):
    return x * (x > 0)

def dReLU(x):
    return 1. * (x > 0)

import random
np.random.seed(42)
#conv1 = np.array([[[10,10,10],[0,0,0],[-10,-10,-10]],[[-10,0,10],[-10,0,10],[-10,0,10]]]).reshape(2,1,3,3)
conv1 = np.random.randn(3,1,3,3) * np.sqrt(1. / 3)
w1 = np.random.rand(60,13*13*3)/np.sqrt(13*13*3)
b0 = np.zeros((60,1))/np.sqrt(13*13*3)
w2 = np.random.rand(10,60)/np.sqrt(60)
b1 = np.zeros((10,1))/np.sqrt(30)
loss=[]
batches = 100

lr = 0.01
batch_size = 200
beta = 0.9
count = 0
epochs = 5
    
X_train_reshape = np.zeros((X_train.shape[1],1,28,28))

for i in range(X_train.shape[1]):
    temp = X_train[:,i]
    temp = np.ravel(temp)
    temp = temp.reshape(28,28)
    X_train_reshape[i,0,:,:] = temp
    
X_train= X_train_reshape  

X_test_reshape = np.zeros((X_test.shape[1],1,28,28))

from matplotlib import pyplot as plt

for i in range(X_test.shape[1]):
    temp = X_test[:,i]
    temp = np.ravel(temp)
    temp = temp.reshape(28,28)
#     #plt.plot(temp)
#     plt.imshow(temp)
#     plt.pause(0.5)
    X_test_reshape[i,0,:,:] = temp

X_test= X_test_reshape  

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
    
    



In [ ]:
batch_size = 100
batches = int(X_train.shape[0]/batch_size)
print(batches)
epochs = 150
lr = 0.001
loss_weight_dict = {
    
}
import time

X_conv = 0

### Implementing Adam Optimizer

beta1 = 0.9
beta2 = 0.995
momentum_w1 = 0
momentum_w2 = 0
momentum_b0 = 0
momentum_b1 = 0
momentum_conv1 = 0
velocity_w1 = 0
velocity_w2 = 0
velocity_b0 = 0
velocity_b1 = 0
velocity_conv1 = 0



### Forward Pass
for i in range(epochs):
    permutation = np.random.permutation(X_train.shape[0])
    X_train_shuffled = X_train[permutation,:,:,:]
    Y_train_shuffled = y_train[:, permutation]
#     beta1 = 0.9
#     beta2 = 0.995
#     momentum_w1 = 0
#     momentum_w2 = 0
#     momentum_b0 = 0
#     momentum_b1 = 0
#     momentum_conv1 = 0
#     velocity_w1 = 0
#     velocity_w2 = 0
#     velocity_b0 = 0
#     velocity_b1 = 0
#     velocity_conv1 = 0

    start = time.process_time()
    for j in range(batches):
        #print(conv1)
        begin = j * batch_size
        
        end = min(begin + batch_size, X_train.shape[0] - 1)
        if begin>=end:
            continue
        X = X_train_shuffled[begin:end,:,:,:]
        Y = Y_train_shuffled[:, begin:end]
        ## Get batch size
        m_batch = end - begin
        ## Pass input through convolutional Layer
        X_im2col = im2col(X=X,conv1=conv1,pad=0,stride=1)
        conv1_col = conv1.reshape(conv1.shape[0],-1)
        X_result = conv1_col@X_im2col
        X_result = np.array(np.hsplit(X_result, m_batch)).reshape((m_batch,conv1.shape[0],26,26))
        X_conv = ReLU(X_result)
        ### Pass output of convolutional layer to maxpooling layer
        X_maxpool,max_index = maxpool(input_matrix=X_conv)
        ### Get shape to flatten the maxpool output
        flattened_shape = X_maxpool.shape[1]*X_maxpool.shape[2]*X_maxpool.shape[3]
        ### Re-shape the maxpool output
        X_maxpool_reshape = maxpool_reshape(shape_=(flattened_shape,m_batch),input_matrix=X_maxpool)
        ## Pass the re-shaped (flattened) layer to sigmoid function
        x1 = ReLU(w1@X_maxpool_reshape+b0)
        ## Pass the output of the sigmoid layer to softmax layer for classification output
        x2 = softmax(w2@x1+b1)
        #print('Time Taken Forward Prop : ',time.process_time()-start)
        ### Calculate Deltas
        delta_2 = (x2-Y)
        #np.multiply(x1,1-x1)
        delta_1 = np.multiply(w2.T@delta_2,dReLU(x1))
        delta_maxpool_reshape = np.multiply(w1.T@delta_1,1)
        delta_maxpool = np.zeros(X_maxpool.shape)
        #print(delta_relu.shape)
        #### Re-shape maxpool delta create function
        #for image in range(delta_maxpool_reshape.shape[1]):
        delta_maxpool[0:m_batch,:,:,:] = delta_maxpool_reshape[:,0:m_batch].ravel().reshape(m_batch,X_maxpool.shape[1],X_maxpool.shape[2],X_maxpool.shape[3])
        #print('Time Taken For Re-shape Maxpool Delta : ',time.process_time()-start)    
        ## Propagate Delta to convolutional Layer from Max Layer
        delta_x_conv = np.zeros((X_conv.shape))
        for p in max_index:
            
            delta_x_conv[p[0],p[1],p[2],p[3]] = delta_maxpool[p[4],p[5],p[6],p[7]]
        
        #print('Time Taken to propagate delta for re-shape max pool delta : ',time.process_time()-start)
        
        dout = np.multiply(delta_x_conv,dReLU(X_conv))
        dout_test= np.zeros((dout.shape[1],dout.shape[0]*dout.shape[2]*dout.shape[3]))
        dout_test = np.zeros((delta_x_conv.shape[1],delta_x_conv.shape[0]*delta_x_conv.shape[2]*delta_x_conv.shape[3]))
        for channel in range(delta_x_conv.shape[1]):
            delta_x_conv_channel = delta_x_conv[:,channel,:,:]
            dout_test[channel:channel+1,:] = np.ravel(delta_x_conv_channel)
        dw_col = dout_test @ X_im2col.T
        #dconv1 = dw_col.reshape((dw_col.shape[0],1,3, 3))
        
        
        
        
        #print('Done!')
        if i==0 :
            dW1 = delta_1@X_maxpool_reshape.T
            dW2 = delta_2@x1.T
            db0 = np.sum(delta_1,axis=1,keepdims=True)
            db1 = np.sum(delta_2,axis=1,keepdims=True)
            dconv1 = dw_col.reshape((dw_col.shape[0],1,3, 3))
            momentum_w1 = beta1*momentum_w1 + ((1-beta1)*dW1)
            momentum_w2 = beta1*momentum_w2 + ((1-beta1)*dW2)
            momentum_b0 = beta1*momentum_b0 + ((1-beta1)*db0)
            momentum_b1 = beta1*momentum_b1 + ((1-beta1)*db1)
            momentum_conv1 = beta1*momentum_conv1 + ((1-beta1)*dconv1)
            velocity_w1 = beta2*velocity_w1 + ((1-beta2)*dW1**2)
            velocity_w2 = beta2*velocity_w2 + ((1-beta2)*dW2**2)
            velocity_b0 = beta2*velocity_b0 + ((1-beta2)*db0**2)
            velocity_b1 = beta2*velocity_b1 + ((1-beta2)*db1**2)
            velocity_conv1 = beta2*velocity_conv1 + ((1-beta2)*dconv1**2)
            
            
            #print('Time Taken to calculate dconv1 : ',time.process_time()-start)
            ### Create dconv1
        else:
            
            ## Create Conv1
            dW1 = delta_1@X_maxpool_reshape.T
            dW2 = delta_2@x1.T
            db0 = np.sum(delta_1,axis=1,keepdims=True)
            db1 = np.sum(delta_2,axis=1,keepdims=True)
            dconv1 = dw_col.reshape((dw_col.shape[0],1,3, 3))
            #print('Time Taken to calculate dconv1 : ',time.process_time()-start)
            momentum_w1 = beta1*momentum_w1 + ((1-beta1)*dW1)
            momentum_w2 = beta1*momentum_w2 + ((1-beta1)*dW2)
            momentum_b0 = beta1*momentum_b0 + ((1-beta1)*db0)
            momentum_b1 = beta1*momentum_b1 + ((1-beta1)*db1)
            momentum_conv1 = beta1*momentum_conv1 + ((1-beta1)*dconv1)
            velocity_w1 = beta2*velocity_w1 + ((1-beta2)*(dW1**2))
            velocity_w2 = beta2*velocity_w2 + ((1-beta2)*(dW2**2))
            velocity_b0 = beta2*velocity_b0 + ((1-beta2)*(db0**2))
            velocity_b1 = beta2*velocity_b1 + ((1-beta2)*(db1**2))
            velocity_conv1 = beta2*velocity_conv1 + ((1-beta2)*(dconv1**2))
            #print(time.process_time()-start)
            


        w1 = w1 - ((lr*momentum_w1)/(np.sqrt(velocity_w1)+0.0000001))
        b0 = b0 - ((lr*momentum_b0)/(np.sqrt(velocity_b0)+0.0000001))
        w2 = w2 - ((lr*momentum_w2)/(np.sqrt(velocity_w2)+0.0000001))
        b1 = b1 - ((lr*momentum_b1)/(np.sqrt(velocity_b1)+0.0000001))
        conv1 = conv1 -((lr*momentum_conv1)/(np.sqrt(velocity_conv1)+0.0000001))
    
    #lr = lr*np.exp(-0.1*i)
    
    X_im2col = im2col(X=X_train,conv1=conv1,pad=0,stride=1)
    conv1_col = conv1.reshape(conv1.shape[0],-1)
    X_result = conv1_col@X_im2col
    X_result = np.array(np.hsplit(X_result, X_train.shape[0])).reshape((X_train.shape[0],conv1.shape[0],26,26))
    X_train_ = ReLU(X_result)    
    #X_train_ = convolution(input_matrix=X_train,filter_=conv1)
    X_train_,max_index = maxpool(input_matrix = X_train_)
    flattened_shape = X_train_.shape[1]*X_train_.shape[2]*X_train_.shape[3]
    X_train_reshape = np.zeros((flattened_shape,X_train_.shape[0]))
    for row in range(X_train_.shape[0]):
        temp = X_train_[row,:,:,:]
        temp = np.ravel(temp)
        X_train_reshape[:,row] = temp
    
    X_train_ = X_train_reshape
    x1 = ReLU(w1@X_train_+b0)
    x2_train = softmax(w2@x1+b1)
    x2_train_df = pd.DataFrame(x2_train)
    x2_train_df = (x2_train_df == x2_train_df.max()).astype(int)
    x2_train_df = x2_train_df.transpose()
    x2_train_df = pd.merge(x2_train_df,one_hot)
    x2_train_df = x2_train_df[['label']]
    y_train_df = pd.merge(pd.DataFrame(y_train.T),one_hot)
    x2_train_df['label_actual'] = y_train_df['label']
    train_accuracy = np.sum(x2_train_df['label_actual']==x2_train_df['label'])/x2_train_df.shape[0]

    
    print('Training Loss...')
    print(-np.mean(np.multiply(y_train,np.log(x2_train))))
    
#     if -np.mean(np.multiply(y_train,np.log(x2_train))) < 0.01:
#         lr = 0.001
#     if -np.mean(np.multiply(y_train,np.log(x2_train))) < 0.005:
#         lr = 0.001
    
    add_loss = {
        'loss' : -np.mean(np.multiply(y_train,np.log(x2_train))),
        'weight_1' : w1,
        'weight_2':w2,
        'conv1':conv1,
        'b0' : b0,
        'b1': b1,
        'train_accuracy': train_accuracy
    }
    
    
    X_im2col = im2col(X=X_test,conv1=conv1,pad=0,stride=1)
    conv1_col = conv1.reshape(conv1.shape[0],-1)
    X_result = conv1_col@X_im2col
    X_result = np.array(np.hsplit(X_result, X_test.shape[0])).reshape((X_test.shape[0],conv1.shape[0],26,26))
    X_test_ = ReLU(X_result)
    
    #X_test_ = convolution(input_matrix=X_test,filter_=conv1)
    X_test_,max_index = maxpool(input_matrix=X_test_)
    flattened_shape = X_test_.shape[1]*X_test_.shape[2]*X_test_.shape[3]
    X_test_reshape = np.zeros((flattened_shape,X_test_.shape[0]))
    for row in range(X_test_.shape[0]):
        temp = X_test_[row,:,:,:]
        temp = np.ravel(temp)
        X_test_reshape[:,row] = temp
    
    X_test_ = X_test_reshape
    x1 = ReLU(w1@X_test_+b0)
    x2_test = softmax(w2@x1+b1)
    x2_test_df = pd.DataFrame(x2_test)
    x2_test_df = (x2_test_df == x2_test_df.max()).astype(int)
    x2_test_df = x2_test_df.transpose()
    x2_test_df = pd.merge(x2_test_df,one_hot)
    x2_test_df = x2_test_df[['label']]
    y_test_df = pd.merge(pd.DataFrame(y_test.T),one_hot)
    x2_test_df['label_actual'] = y_test_df['label']
    test_accuracy = np.sum(x2_test_df['label_actual']==x2_test_df['label'])/x2_test_df.shape[0]
    print('Epoch: ',i)
    print('Time Taken for Epoch', i)
    print(time.process_time() - start)
    print('Testing Accuracy :',test_accuracy)
    print('Training Accuracy :',train_accuracy)
    print('----------------------------------------')
    
    
    
#     print('Testing Loss...')
#     print(-np.mean(np.multiply(y_test,np.log(x2))))
    
    add_loss['testing_loss'] = -np.mean(np.multiply(y_test,np.log(x2_test)))
    add_loss['test_accuracy'] = test_accuracy
    loss_weight_dict[count] = add_loss
    count = count + 1


In [ ]:
loss = []

for i in range(len(loss_weight_dict)):
    loss.append(loss_weight_dict[i]['loss'])
import matplotlib.pyplot as plt
plt.plot(loss)
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.show()

In [ ]:
train_accuracy = []

for i in range(len(loss_weight_dict)):
    train_accuracy.append(loss_weight_dict[i]['train_accuracy'])
import matplotlib.pyplot as plt
plt.plot(train_accuracy)
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.show()

In [ ]:
test_accuracy = []

for i in range(len(loss_weight_dict)):
    test_accuracy.append(loss_weight_dict[i]['test_accuracy'])
import matplotlib.pyplot as plt
plt.plot(test_accuracy)
plt.xlabel('Epochs')
plt.ylabel('Testing Accuracy')
plt.show()
